# 1. Tester et évaluer un modèle entraîné sur Google News

## **a.** 

Installez gensim, une librairie Python qui fournit des outils pour travailler avec Word2Vec (avec conda ou avec pip).**

Prenez la version 3.8.3, et non pas la nouvelle version 4.0.X. Obtenez depuis gensim le modèle word2vec pré-entraîné sur le corpus Google News en écrivant : 
```python 
w2v_model = gensim.downloader.load("word2vec-google-news-300")
```
Ce qui téléchargera le fichier la première fois. Ne gardez en mémoire que les vecteurs des mots, en écrivant :  

```python 
w2v_vectors = w2v_model.wv
# puis
del w2v_model
``` 
• Une fois que vous avez téléchargé le modèle, vous pouvez utiliser votre copie locale :
```python 
w2v_vectors = KeyedVectors.load_word2vec_format(path_to_file, binary=True)
```


In [58]:
%pip install gensim


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [59]:
from gensim import downloader, models
# downloader.load("word2vec-google-news-300")

In [1]:
from gensim.models import KeyedVectors

path_to_file = "~/gensim-data/word2vec-google-news-300/word2vec-google-news-300.gz"
w2v_vectors = KeyedVectors.load_word2vec_format(path_to_file, binary=True)

## **b.** 
Quelle place mémoire occupe le processus du notebook une fois les vecteurs de mots chargés ?

Le processus python du notebook occupe **3.92 GB** de mémoire une fois les vecteurs de mots chargés.

## **c.** 

Quelle est la dimension de l’espace vectoriel dans lequel les mots sont représentés ?

In [61]:
size_vocab, nb_dimension = w2v_vectors.vectors.shape

print(f'Nombre de dimensions: {nb_dimension}')

Nombre de dimensions: 300


## **d.**

Quelle est la taille du vocabulaire du modèle ? Affichez cinq mots (anglais) qui sont dans le vocabulaire et deux qui ne le sont pas.

In [62]:
# d. Quelle est la taille du vocabulaire du modèle ? Affichez cinq mots (anglais) qui sont dans le vocabulaire et deux qui ne le sont pas.

print(f'Taille du vocabulaire : {size_vocab}')

words = ['hello', 'world', 'apple', 'dog', 'cat', 'cryptocurrency', 'deepfake']
for word in words:
    print(f'{word} : {word in w2v_vectors.key_to_index}')


Taille du vocabulaire : 3000000
hello : True
world : True
apple : True
dog : True
cat : True
cryptocurrency : False
deepfake : False


## **e.** 

Quelle est la distance entre les mots rabbit et carrot ? Veuillez aussi expliquer en une phrase comment on mesurer les distances entre deux mots dans cet espace.

La distance entre deux mots est mesurée en utilisant la similarité cosinus. En effet, la distance correspond à 1 - la similarité.

In [63]:
rabbit_carrot_dist = w2v_vectors.distance('rabbit', 'carrot')

print(f'Distance entre rabbit et carrot : {rabbit_carrot_dist}')

Distance entre rabbit et carrot : 0.6369356513023376


## **f.** 
Considérez au moins 5 paires de mots, certains proches par leurs sens, d’autres plus éloignés. Pour chaque paire, calculez la distance entre les deux mots. Veuillez indiquer si les distances obtenues correspondent à vos intuitions sur la proximité des sens des mots.

In [64]:
word_pairs = [('festival', 'watch'), ('confident', 'cocky'), ('tree', 'leaf'), ('meticulous', 'nitpicky'), ('dog', 'car')]
for pair in word_pairs:
    print(f'Distance entre {pair[0]} et {pair[1]} : {w2v_vectors.distance(pair[0], pair[1])}')

Distance entre festival et watch : 0.9337479248642921
Distance entre confident et cocky : 0.6400514245033264
Distance entre tree et leaf : 0.5177146792411804
Distance entre meticulous et nitpicky : 0.7041851878166199
Distance entre dog et car : 0.68999844789505


Certaines paires obtiennent des distances qui correspondent à nos intuitions, d'autres non. Par exemple, les paires (festival, watch) et (dog, car) ont une distance élevée car les mots n'ont pas de sens proche. 

Par contre, d'autres paires comme (tree, leaf), (confident, cocky) ou (meticulous, nitpicky) ont une distance plus élevée que ce que nous aurions pensé.

## **g.** 

Pouvez-vous trouver des mots de sens opposés mais qui sont proches dans l’espace vectoriel ? Comment expliquez vous cela ? Est-ce une qualité ou un défaut du modèle word2vec ?

In [65]:
opposite_pairs = [('good', 'bad'), ('up', 'down'), ('black', 'white')]

for pair in opposite_pairs:
    print(f'Distance entre {pair[0]} et {pair[1]} : {w2v_vectors.distance(pair[0], pair[1])}')

Distance entre good et bad : 0.28099489212036133
Distance entre up et down : 0.36030077934265137
Distance entre black et white : 0.19077849388122559


Cela est dû au fait que les mots peuvent être utilisés dans le même contexte et possèdent un lien sémantique. Il s'agit d'une qualité du modèle car il arrive alors à faire des rapprochements plus subtiles que simplement trouver des similarités entre synonymes par exemple.

## **h.** 

En vous aidant de la documentation de Gensim sur KeyedVectors, calculez le score du modèle word2vec sur les données WordSimilarity-353. (La doc vous permettra aussi de récupérer le fichier.)  

Expliquez en 1-2 phrases comment ce score est calculé et ce qu’il mesure.

In [3]:
from gensim.test.utils import datapath

eval_wordsim = w2v_vectors.evaluate_word_pairs(datapath('wordsim353.tsv'))
print("Wordsim353:")
print(eval_wordsim)

Wordsim353:
(PearsonRResult(statistic=0.6238773487289394, pvalue=1.7963224351224885e-39), SignificanceResult(statistic=0.6589215888009288, pvalue=2.534605645914962e-45), 0.0)


La méthode `evaluate_word_pairs()` compare la similarité cosinus entre les vecteurs de mots du modèle avec les scores de similarité humaine fournis dans le fichier `wordsim353.tsv`. Le score mesuré est la corrélation de Spearman entre ces deux ensembles de scores de similarité, et il indique dans quelle mesure le modèle est capable de reproduire les jugements de similarité humaine sur ces paires de mots spécifiques.

## **i.** 

En vous aidant de la documentation, calculez le score du modèle word2vec sur les données `questions-words.txt`. Attention, cette évaluation prend une dizaine de minutes. Expliquez en 1-2 phrases comment ce score est calculé et ce qu’il mesure.

In [4]:
eval_qwords = w2v_vectors.evaluate_word_analogies(
    datapath('questions-words.txt'))

In [7]:
print("Questions Words:")
print(f"Score: {eval_qwords[0]}")

Questions Words:
Score: 0.7401448525607863


La méthode `evaluate_word_analogies()` évalue la performance du modèle sur des tâches d'analogie de mots, où il doit trouver un mot D qui complète l'analogie A est à B comme C est à D. Le score mesuré est la proportion de bonnes réponses parmi toutes les analogies testées, indiquant la capacité du modèle à résoudre correctement ces analogies en utilisant ses représentations vectorielles de mots.

# 2. Entraîner deux nouveaux modèles word2vec à partir de nouveaux corpus

## **a.** 

En utilisant gensim.downloader, récupérez le corpus qui contient les 108 premiers caractères de Wikipédia (en anglais) avec la commande :**
```python
corpus = gensim.downloader.load('text8')
```

Combien de phrases et de mots (tokens) possède ce corpus ?

In [69]:
corpus = downloader.load('text8')

print("Number of sentences", downloader.info('text8')['num_records'])
print("Number of words: ", sum(len(i) for i in corpus))

Number of sentences 1701
Number of words:  17005207


## **b.** 

Entraînez un nouveau modèle word2vec sur ce nouveau corpus. Si nécessaire, procédez progressivement, en commençant par 1% du corpus, puis 10%, pour contrôler le temps nécessaire. 

• Indiquez la dimension choisie pour le embedding de ce nouveau modèle.  
• Combien de temps prend l’entraînement sur le corpus total ?  
• Quelle est la taille (en Mo) du modèle word2vec résultant ?

In [70]:
from gensim.models import Word2Vec
import time

corpus_1 = [sentence[:len(sentence)//100] for sentence in corpus]
corpus_10 = [sentence[:len(sentence)//10] for sentence in corpus]

# 1% du corpus
start_time = time.time()
model_1 = Word2Vec(corpus_1)
print(f"Temps d'entrainement sur 1% du corpus : {time.time() - start_time}")

# 10% du corpus
start_time = time.time()
model_10 = Word2Vec(corpus_10)
print(f"Temps d'entrainement sur 10% du corpus : {time.time() - start_time}")

# 100% du corpus
start_time = time.time()
model_100 = Word2Vec(corpus)
print(f"Temps d'entrainement sur 100% du corpus : {time.time() - start_time}")

Temps d'entrainement sur 1% du corpus : 0.2372300624847412
Temps d'entrainement sur 10% du corpus : 2.9472970962524414
Temps d'entrainement sur 100% du corpus : 36.639487981796265


In [71]:
print(
    f"Taille du modèle sur 1% du corpus : {model_1.estimate_memory()['total'] / 1000000} Mo, Dimension des embeddings: {model_1.vector_size}")
print(
    f"Taille du modèle sur 10% du corpus : {model_10.estimate_memory()['total'] / 1000000} Mo, Dimension des embeddings: {model_10.vector_size}")
print(
    f"Taille du modèle sur 100% du corpus : {model_100.estimate_memory()['total'] / 1000000} Mo, Dimension des embeddings: {model_100.vector_size}")

Taille du modèle sur 1% du corpus : 5.4054 Mo, Dimension des embeddings: 100
Taille du modèle sur 10% du corpus : 25.9779 Mo, Dimension des embeddings: 100
Taille du modèle sur 100% du corpus : 92.677 Mo, Dimension des embeddings: 100


## **c.** 

Mesurez la qualité de ce modèle comme dans la partie 1, points i et j. Ce modèle est-il meilleur que celui entraîné sur Google News ? Quelle serait la raison de la différence ?


In [72]:
eval_qwords_text8 = model_100.wv.evaluate_word_analogies(datapath('questions-words.txt'))
eval_wordsim_text8 = model_100.wv.evaluate_word_pairs(datapath('wordsim353.tsv'))

In [82]:
print("Wordsim353:")
print("Google News: ", eval_wordsim[0])
print("Text8: ", eval_wordsim_text8[0])

print("\nQuestions Words:")
print("Google News:\t", eval_qwords[0])
print("Text8:\t\t", eval_qwords_text8[0])


Wordsim353:
Google News:  PearsonRResult(statistic=0.6238773487289394, pvalue=1.7963224351224885e-39)
Text8:  PearsonRResult(statistic=0.6191131746160552, pvalue=1.6004556742509145e-38)

Questions Words:
Google News:	 0.7401448525607863
Text8:		 0.23621473046502497


In [74]:
# Différences entre les modèles
w2v_vec_tokens, w2v_vec_dim = w2v_vectors.vectors.shape
model_100_tokens, model_100_dim = model_100.wv.vectors.shape

print(f'Nombre de tokens dans le modèle Google News (dimension de {w2v_vec_dim}):\t {w2v_vec_tokens}')
print(f'Nombre de tokens dans le modèle Text8 (dimension de {model_100_dim}):\t {model_100_tokens}')

Nombre de tokens dans le modèle Google News (dimension de 300):	 3000000
Nombre de tokens dans le modèle Text8 (dimension de 100):	 71290


Le modèle Google News est bien plus performant que notre modèle entraîné sur le corpus Text8. On observe que son score de similarité est trois fois plus élevé que celui de Text8 lors de l'évaluation `Questions Words`. Pour l'évaluation `Wordsim353`, elle est légèrement plus élevée avec une pvalue également plus petite, indiquant une probabilité moins élevée que ce soit le fruit du hasard.

Nous supposons que la taille du corpus de Google News est bien supérieure à celle de Text8 culminant à environ 17 millions de mots. De plus, nous savons que la taille du vocabulaire du modèle Google News est de 3 millions de mots, contre 71'290 pour Text8, soit 42x fois plus grande. Il est donc normal que le modèle Google News soit plus performant que le modèle Text8.

## **d.** 

Téléchargez maintenant le corpus quatre fois plus grand constitué de la concaténation du corpus text8 et des dépêches économiques de Reuters (413 Mo) fourni en ligne par l’enseignant et appelé wikipedia_augmented.dat. Entraînez un nouveau modèle word2vec sur ce corpus, en précisant la dimension du plongement (embedding).


• Utilisez la classe `Text8Corpus()` pour charger le corpus et faire la tokenization et la segmentation en phrases.  
• Combien de temps prend l’entraînement ?  
• Quelle est la taille (en Mo) du modèle word2vec résultant ?


In [83]:
from gensim.models.word2vec import Text8Corpus

corpus_augmented = Text8Corpus('wikipedia_augmented.dat')

# model_augmented = models.Word2Vec(corpus_augmented)
# model_augmented.save("wiki_augmented.model")

# model_augmented_300 = models.Word2Vec(corpus_augmented, vector_size=300)
# model_augmented_300.save("wiki_augmented_300.model")

Temps d'entraînement avec une dimension de 300 : 4m 17s

In [84]:
model_augmented = models.Word2Vec.load("wiki_augmented.model")
model_augmented_300 = models.Word2Vec.load("wiki_augmented_300.model")

In [86]:
print(
    f"Taille du modèle augmenté : {model_augmented.estimate_memory()['total'] / 1000000} Mo, Dimension des embeddings: {model_augmented.vector_size}")

print(
    f"Taille du modèle augmenté de dimension 300 : {model_augmented_300.estimate_memory()['total'] / 1000000} Mo, Dimension des embeddings: {model_augmented_300.vector_size}")


Taille du modèle augmenté : 161.6836 Mo, Dimension des embeddings: 100
Taille du modèle augmenté de dimension 300 : 360.6788 Mo, Dimension des embeddings: 300


## **e.** 

Testez ce modèle comme en 1.h et 1.i. Est-il meilleur que le précédent ? Pour quelle raison ?

In [76]:
eval_qwords_text8_augmented = model_augmented.wv.evaluate_word_analogies(datapath('questions-words.txt'))
eval_wordsim_text8_augmented = model_augmented.wv.evaluate_word_pairs(datapath('wordsim353.tsv'))

In [87]:
eval_qwords_text8_augmented_300 = model_augmented_300.wv.evaluate_word_analogies(
    datapath('questions-words.txt'))
eval_wordsim_text8_augmented_300 = model_augmented_300.wv.evaluate_word_pairs(
    datapath('wordsim353.tsv'))

In [88]:
print("Wordsim353:")
print("Google News: ", eval_wordsim[0])
print("Text8: ", eval_wordsim_text8[0])
print("Text8 Augmented: ", eval_wordsim_text8_augmented[0])
print("Text8 Augmented 300: ", eval_wordsim_text8_augmented_300[0])

print("\nQuestions Words:")
print("Google News:\t\t", eval_qwords[0])
print("Text8:\t\t\t", eval_qwords_text8[0])
print("Text8 Augmented:\t", eval_qwords_text8_augmented[0])
print("Text8 Augmented 300:\t", eval_qwords_text8_augmented_300[0])


Wordsim353:
Google News:  PearsonRResult(statistic=0.6238773487289394, pvalue=1.7963224351224885e-39)
Text8:  PearsonRResult(statistic=0.6191131746160552, pvalue=1.6004556742509145e-38)
Text8 Augmented:  PearsonRResult(statistic=0.4970035845305565, pvalue=2.019254963728344e-23)
Text8 Augmented 300:  PearsonRResult(statistic=0.5106467378462217, pvalue=7.749511810213081e-25)

Questions Words:
Google News:		 0.7401448525607863
Text8:			 0.23621473046502497
Text8 Augmented:	 0.2873069393082445
Text8 Augmented 300:	 0.3540896236676093


In [78]:
model_augmented_tokens, model_augmented_dim = model_augmented.wv.vectors.shape

print(
    f'Nombre de tokens dans le modèle Google News (dimension de {w2v_vec_dim}):\t\t {w2v_vec_tokens}')
print(
    f'Nombre de tokens dans le modèle Text8 (dimension de {model_100_dim}):\t\t {model_100_tokens}')
print(
    f'Nombre de tokens dans le modèle Text8 Augmented (dimension de {model_augmented_dim}):\t {model_augmented_tokens}')


Nombre de tokens dans le modèle Google News (dimension de 300):		 3000000
Nombre de tokens dans le modèle Text8 (dimension de 100):		 71290
Nombre de tokens dans le modèle Text8 Augmented (dimension de 100):	 124372


Les modèles augmentés sont meilleurs sur `Questions Words` que le modèle Text8 de base, mais toujours beaucoup moins bons que le modèle Google News. On observe que le nombre de tokens dans le modèle augmenté a presque doublé par rapport au modèle Text8 de base, il semble que la taille du corpus puis du vocabulaire soient donc des facteurs influençant la performance du modèle. Le score de similarité sur `Wordsim353` est par contre moins bon que celui du modèle Text8 de base et possède une pvalue plus élevée, indiquant une corrélation moins forte. On suppose que le mélange de deux corpus différents peut avoir un impact négatif sur la corrélation.

Nous avons également entrainé un modèle avec une dimension de 300 pour ses embeddings pour se rapprocher des paramètres du modèle Google News. Cela a également permis d'augmenter la performance du modèle sur `Questions Words` et a légèrement mitigé la perte de performances sur `Wordsim353`.